In [1]:
from segger.data.parquet.sample import STSampleParquet
from path import Path
from segger.data.utils import calculate_gene_celltype_abundance_embedding
import scanpy as sc
from segger.data.parquet._utils import get_polygons_from_xy
from segger.data.parquet.sample import STSampleParquet
from segger.training.segger_data_module import SeggerDataModule
from segger.training.train import LitSegger
#from segger.prediction.predict_parquet import segment, load_model
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch import Trainer
from pathlib import Path
import pandas as pd
from segger.models.segger_model import Segger
from torch_geometric.nn import to_hetero


SCRNASEQ_FILE='/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/xenium_colon/scRNAseq.h5ad'
XENIUM_DATA_DIR='/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/xenium_colon'
CELLTYPE_COLUMN = "Level1"

# Base directory to store Pytorch Lightning models
xenium_data_dir = '/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/xenium_colon'
segger_data_dir = '/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/segger_colon'

scrnaseq = sc.read(SCRNASEQ_FILE)
sc.pp.subsample(scrnaseq, 0.1)
scrnaseq.var_names_make_unique()
# Calculate gene-celltype embeddings from reference data
gene_celltype_abundance_embedding = calculate_gene_celltype_abundance_embedding(
    scrnaseq,
    CELLTYPE_COLUMN
) 

sample = STSampleParquet(
    base_dir=XENIUM_DATA_DIR,
    n_workers=4,
    sample_type="xenium",
    # scale_factor=0.8,
    weights=gene_celltype_abundance_embedding
)

# only run once if you want to create the dataset
# sample.save(
#       data_dir=segger_data_dir,
#       k_bd=3,
#       dist_bd=15.0,
#       k_tx=3,
#       dist_tx=5.0,
#       tile_width=120,
#       tile_height=120,
#       neg_sampling_ratio=5.0,
#       frac=1.0,
#       val_prob=0.1,
#       test_prob=0.2,
# )

/home/f833u/fySegger/lib/python3.11/site-packages/anndata/_core/anndata.py:1776: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [ ]:
# Base directory to store Pytorch Lightning models
models_dir = '/dkfz/cluster/gpu/checkpoints/OE0606/fengyun/segger_model/segger_colon'

# Initialize the Lightning data module
dm = SeggerDataModule(
    data_dir=segger_data_dir,
    batch_size=3,
    num_workers=2,
)

dm.setup()

# num_tx_tokens = 500

# If you use custom gene embeddings, use the following two lines instead:
num_tx_tokens = dm.train[0].x_dict["tx"].shape[1] # Set the number of tokens to the number of genes
print(f"Number of tokens: {num_tx_tokens}")


model = Segger(
    num_tx_tokens=num_tx_tokens,
    init_emb=8,
    hidden_channels=64,
    out_channels=16,
    heads=4,
    num_mid_layers=3,
)
model = to_hetero(model, (["tx", "bd"], [("tx", "belongs", "bd"), ("tx", "neighbors", "tx")]), aggr="sum")



Number of tokens: 9


In [ ]:
batch = dm.train[0]
model.forward(batch.x_dict, batch.edge_index_dict)
# Wrap the model in LitSegger
ls = LitSegger(model=model)

# Initialize the Lightning trainer
trainer = Trainer(
    #accelerator='auto', # 'gpu' or 'cpu'
    accelerator='auto',
    strategy='auto',
    precision='16-mixed',
    devices=1, # set higher number if more gpus are available
    max_epochs=100,
    default_root_dir=models_dir,
    logger=CSVLogger(models_dir),
)

# Fit model
trainer.fit(
    model=ls,
    datamodule=dm
)

MisconfigurationException: `CUDAAccelerator` can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].

In [1]:
from segger.data.parquet.sample import STSampleParquet
from path import Path
from segger.data.utils import calculate_gene_celltype_abundance_embedding
import scanpy as sc
from segger.data.parquet._utils import get_polygons_from_xy
from segger.data.parquet.sample import STSampleParquet
from segger.training.segger_data_module import SeggerDataModule
from segger.training.train import LitSegger
#from segger.prediction.predict_parquet import segment, load_model
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch import Trainer
from pathlib import Path
import pandas as pd
from segger.models.segger_model import Segger
from torch_geometric.nn import to_hetero


# SCRNASEQ_FILE='/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/xenium_colon/scRNAseq.h5ad'
XENIUM_DATA_DIR='/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/xenium_colon'
# CELLTYPE_COLUMN = "Level1"

# Base directory to store Pytorch Lightning models
xenium_data_dir = '/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/xenium_colon'
segger_data_dir = '/dkfz/cluster/gpu/data/OE0606/fengyun/xenium_data/segger_colon_no_seq'

# scrnaseq = sc.read(SCRNASEQ_FILE)
# sc.pp.subsample(scrnaseq, 0.1)
# scrnaseq.var_names_make_unique()
# # Calculate gene-celltype embeddings from reference data
# gene_celltype_abundance_embedding = calculate_gene_celltype_abundance_embedding(
#     scrnaseq,
#     CELLTYPE_COLUMN
# )


sample = STSampleParquet(
    base_dir=XENIUM_DATA_DIR,
    n_workers=4,
    sample_type="xenium",
    # scale_factor=0.8,
    # weights=gene_celltype_abundance_embedding
)

# only run once if you want to create the dataset
# sample.save(
#       data_dir=segger_data_dir,
#       k_bd=3,
#       dist_bd=15.0,
#       k_tx=3,
#       dist_tx=5.0,
#       tile_width=120,
#       tile_height=120,
#       neg_sampling_ratio=5.0,
#       frac=1.0,
#       val_prob=0.1,
#       test_prob=0.2,
# )



In [2]:
# Base directory to store Pytorch Lightning models
models_dir = '/dkfz/cluster/gpu/checkpoints/OE0606/fengyun/segger_model/segger_colon_no_seq'

# Initialize the Lightning data module
dm = SeggerDataModule(
    data_dir=segger_data_dir,
    batch_size=2,
    num_workers=2,
)

dm.setup()

num_tx_tokens = 500

# If you use custom gene embeddings, use the following two lines instead:
# num_tx_tokens = dm.train[0].x_dict["tx"].shape[1] # Set the number of tokens to the number of genes
# print(f"Number of tokens: {num_tx_tokens}")


model = Segger(
    num_tx_tokens=num_tx_tokens,
    init_emb=8,
    hidden_channels=64,
    out_channels=16,
    heads=4,
    num_mid_layers=3,
)
model = to_hetero(model, (["tx", "bd"], [("tx", "belongs", "bd"), ("tx", "neighbors", "tx")]), aggr="sum")

batch = dm.train[0]
model.forward(batch.x_dict, batch.edge_index_dict)
# Wrap the model in LitSegger
ls = LitSegger(model=model)

# Initialize the Lightning trainer
trainer = Trainer(
    accelerator='auto', # 'gpu' or 'cpu'
    #accelerator='cuda',
    strategy='auto',
    precision='16-mixed',
    devices=1, # set higher number if more gpus are available
    max_epochs=100,
    default_root_dir=models_dir,
    logger=CSVLogger(models_dir),
)

# Fit model
trainer.fit(
    model=ls,
    datamodule=dm
)

IndexError: range object index out of range